In [1]:
import uproot

In [9]:
import uproot
import awkward
import numpy as np
from scipy.sparse import csr_matrix, find
from scipy.spatial import cKDTree
from tqdm import tqdm_notebook as tqdm


import os
import os.path as osp

print(os.environ['GNN_TRAINING_DATA_ROOT'])

#fname = osp.join(os.environ['GNN_TRAINING_DATA_ROOT'], 'single_taus/root/partGun_PDGid15_x1000_Pt3.0To100.0_NTUP_1.root')
fname = osp.join(os.environ['GNN_TRAINING_DATA_ROOT'], 'single_taus/root/partGun_PDGid15_x1000_Pt3.0To100.0_NTUP_9.root')

print(type(fname))

test = uproot.open(fname)['ana']['hgc']

%load_ext autoreload
%autoreload 2

/home/lagray/training_data
<class 'str'>
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
test.keys()

[b'event',
 b'lumi',
 b'run',
 b'vtx_x',
 b'vtx_y',
 b'vtx_z',
 b'genpart_eta',
 b'genpart_phi',
 b'genpart_pt',
 b'genpart_energy',
 b'genpart_dvx',
 b'genpart_dvy',
 b'genpart_dvz',
 b'genpart_fbrem',
 b'genpart_pid',
 b'genpart_gen',
 b'genpart_reachedEE',
 b'genpart_fromBeamPipe',
 b'genpart_posx',
 b'genpart_posy',
 b'genpart_posz',
 b'rechit_eta',
 b'rechit_phi',
 b'rechit_pt',
 b'rechit_energy',
 b'rechit_x',
 b'rechit_y',
 b'rechit_z',
 b'rechit_time',
 b'rechit_thickness',
 b'rechit_layer',
 b'rechit_wafer_u',
 b'rechit_wafer_v',
 b'rechit_cell_u',
 b'rechit_cell_v',
 b'rechit_detid',
 b'rechit_isHalf',
 b'rechit_flags',
 b'rechit_cluster2d',
 b'cluster2d_eta',
 b'cluster2d_phi',
 b'cluster2d_pt',
 b'cluster2d_energy',
 b'cluster2d_x',
 b'cluster2d_y',
 b'cluster2d_z',
 b'cluster2d_layer',
 b'cluster2d_nhitCore',
 b'cluster2d_nhitAll',
 b'cluster2d_multicluster',
 b'cluster2d_rechits',
 b'cluster2d_rechitSeed',
 b'multiclus_eta',
 b'multiclus_phi',
 b'multiclus_pt',
 b'multicl

In [30]:
from scipy.sparse import coo_matrix # to encode the cluster mappings
from sklearn.neighbors import NearestNeighbors
from datasets.graph import Graph
from datasets.graph import graph_to_sparse, save_graph

sim_fractions = awkward.fromiter(test['simcluster_fractions'].array())
sim_indices = awkward.fromiter(test['simcluster_hits_indices'].array())
sim_fractions = sim_fractions[sim_indices > -1].compact()
sim_indices = sim_indices[sim_indices > -1].compact()
sim_energy = test['simcluster_energy'].array()
sim_pid = test['simcluster_pid'].array()
sim_eta = test['simcluster_eta'].array()

rechit_layer = test['rechit_layer'].array()
rechit_time = test['rechit_time'].array()
rechit_energy = test['rechit_energy'].array()

rechit_x = test['rechit_x'].array()
rechit_y = test['rechit_y'].array()
rechit_z = test['rechit_z'].array()

In [43]:
def get_features(ievt, mask):
    x = rechit_x[ievt][mask]
    y = rechit_y[ievt][mask]
    z = rechit_z[ievt][mask]
    time = rechit_time[ievt][mask]
    energy = rechit_energy[ievt][mask]    
    return np.stack((x,y,z,time,energy)).T.astype(np.float32)

for i in tqdm(range(rechit_z.size),desc='events processed'): #
        
    #if i > 20: break    
        
    #print(i,sim_pid[i])
    #print(i,sim_eta[i])
    #print(cluster_cats)
        
    sim_indices_cpt = awkward.fromiter(sim_indices[i])
    sim_fractions_cpt = awkward.fromiter(sim_fractions[i])
    if isinstance(sim_indices_cpt, np.ndarray):
        if sim_indices_cpt.size == 0: #skip events that are all noise, they're meaningless anyway
            continue
        else:
            sim_indices_cpt = awkward.JaggedArray.fromcounts([sim_indices_cpt.size], sim_indices_cpt)
            sim_fractions_cps = awkward.JaggedArray.fromcounts([sim_fractions_cpt.size], sim_fractions_cpt)
    fractions_in_clus = sim_fractions_cpt.flatten()
    hits_in_clus = sim_indices_cpt.flatten()
    hit_to_clus = sim_indices_cpt.parents
    
    #print('fracs ',fractions_in_clus.size)
    #print('hitidx',hits_in_clus.size)
    #print('cluidx',hit_to_clus.size)

    
    fracs_to_hits = csr_matrix((fractions_in_clus, (hits_in_clus, hit_to_clus)),
                               (rechit_z[i].size, sim_pid[i].size))    
    
    #print(fracs_to_hits.shape)
    #print(rechit_energy[i].shape)
    
    dense_fracs = fracs_to_hits.todense()
    
    
    pos_clus = (sim_eta[i] > 0)
    neg_clus = ~pos_clus
    
    pos_hits = (rechit_z[i] > 0)
    neg_hits = ~pos_hits
    
    pos_feats = get_features(i, pos_hits)
    neg_feats = get_features(i, neg_hits)
    
    pos_clu_fracs = dense_fracs[:,pos_clus][pos_hits]
    neg_clu_fracs = dense_fracs[:,neg_clus][neg_hits]

    pos_energies = rechit_energy[i][pos_hits]
    neg_energies = rechit_energy[i][neg_hits]
    
    y_gravnet_pos = np.concatenate((pos_energies.reshape((pos_energies.size,1)), 
                                    pos_clu_fracs), axis = -1)
    
    y_gravnet_neg = np.concatenate((neg_energies.reshape((neg_energies.size,1)), 
                                    neg_clu_fracs), axis = -1)
        
    #print(y_gravnet_pos)
    #print(y_gravnet_neg)
    
    pos_graph = Graph(pos_feats, csr_matrix((0,0)), csr_matrix((0,0)), y_gravnet_pos, simmatched = np.array([]))
    #print(np.unique(pos_y,return_counts=True))
    neg_graph = Graph(neg_feats, csr_matrix((0,0)), csr_matrix((0,0)), y_gravnet_neg, simmatched = np.array([]))
    #print(np.unique(neg_y,return_counts=True))
    
    outbase = fname.split('/')[-1].replace('.root','')
    outdir = "/".join(fname.split('/')[:-2]) + "/GravNetTest" + "/raw"
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # for UnnormalizedEdgeNet
    save_graph(pos_graph, '%s/%s_hgcal_gravnet_pos_evt%d.npz'%(outdir,outbase,i))
    save_graph(neg_graph, '%s/%s_hgcal_gravnet_neg_evt%d.npz'%(outdir,outbase,i))
        

In [38]:
%matplotlib inline

import matplotlib.pyplot as plt

def draw_sample(X, Ri, Ro, y, out,
                cmap='bwr_r', 
                skip_false_edges=True,
                alpha_labels=False, 
                sim_list=None): 
    
    #let's draw only the non-noise edges
    out_mask = out > 0
    Ri = Ri[out_mask]
    Ro = Ro[out_mask]
    good_outs = out[out_mask]
    
    # Select the i/o node features for each segment    
    feats_o = X[Ro]
    feats_i = X[Ri]    
    # Prepare the figure
    fig, (ax0,ax1) = plt.subplots(1, 2, figsize=(20,12))
    cmap = plt.get_cmap(cmap)
    
    
    #if sim_list is None:    
        # Draw the hits (layer, x, y)
    #    ax0.scatter(X[:,0], X[:,2], c='k')
    #    ax1.scatter(X[:,1], X[:,2], c='k')
    #else:        
    ax0.scatter(X[:,0], X[:,2], c='k')
    ax1.scatter(X[:,1], X[:,2], c='k')
        
    # Draw the segments    
    if out is not None:
        #t = tqdm.tqdm()
        color_map = {1: dict(c='blue'),
                     2: dict(c='red'),
                     3: dict(c='orange')}
        for j in range(good_outs.shape[0]):
            seg_args = color_map[out[j]]            

            ax0.plot([feats_o[j,0], feats_i[j,0]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
            ax1.plot([feats_o[j,1], feats_i[j,1]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
    else:
        t = tqdm.tqdm(range(y.shape[0]))
        for j in t:
            if y[j]:
                seg_args = dict(c='b', alpha=0.4)
            elif not skip_false_edges:
                seg_args = dict(c='black', alpha=0.4)
            else: continue
                
            ax0.plot([feats_o[j,0], feats_i[j,0]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
            ax1.plot([feats_o[j,1], feats_i[j,1]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
        
    # Adjust axes
    ax0.set_xlabel('$x$ [cm]')
    ax1.set_xlabel('$y$ [cm]')
    ax0.set_ylabel('$layer$ [arb]')
    ax1.set_ylabel('$layer$ [arb]')
    plt.tight_layout()
    return fig;

In [ ]:
X = pos_graph.x.numpy()
index = pos_graph.edge_index.numpy().T
Ro = index[:,0]
Ri = index[:,1]
y = data.y.numpy()

out = np.argmax(pred_edges_np,axis=-1)

draw_sample(X, Ri, Ro, y, y)